In [ ]:
#Importar módulos 
import requests
import json
from urllib.parse import quote
import pandas as pd
from datetime import datetime as dt

In [ ]:
# Parámetros de conexión
HEADERS = {
    'Host': 'www.bloomberg.com',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36',
    'Accept': '*/*',
    'Accept-Language': 'de,en-US;q=0.7,en;q=0.3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'application/json',
    'Content-Type': 'application/json; charset=utf8',
    'media_type': 'application/json',
    'Referer': 'https://www.bloomberg.com/quote/',
    'DNT': '1',
    'Connection': 'keep-alive',
    'TE': 'Trailers'
}
URL_ROOT = 'https://www.bloomberg.com/markets2/api/datastrip'
URL_PARAMS = 'locale=en&customTickerList=true'
VALID_TYPE = {'currency', 'index'}

In [ ]:
# Función Scraper
def scraper(object_id: str = None, object_type: str = None, timeout: int = 5) -> list:
    """
    Get the Bloomberg data for the given object.
    :param object_id: The Bloomberg identifier of the object.
    :param object_type: The type of the object. (Currency or Index)
    :param timeout: Maximal number of seconds to wait for a response.
    :return: The data formatted as dictionary.
    """
    object_type = object_type.lower()
    if object_type not in VALID_TYPE:
        return list()
    # Build headers and url
    object_append = '%s:%s' % (object_id, 'IND' if object_type == 'index' else 'CUR')
    headers = HEADERS
    headers['Referer'] += object_append
    url = '%s/%s?%s' % (URL_ROOT, quote(object_append), URL_PARAMS)
    # Make the request and check response status code
    response = requests.get(url=url, headers=headers)
    if response.status_code in range(200, 230):
        return  response.json()
    return list()

In [ ]:
object_append = '%s:%s' % (object_id, 'IND' if object_type == 'index' else 'CUR')
headers = HEADERS
headers['Referer'] += object_append
url = '%s/%s?%s' % (URL_ROOT, quote(object_append), URL_PARAMS)
response = requests.get(url=url, headers=headers)
response.headers["content-type"]

'text/html, text/html; charset=utf-8'

In [ ]:
#Definimos la moneda 
object_id, object_type = 'PEN', 'currency'
data = scraper(object_id=object_id, object_type=object_type)
print('The exchange rate for USD per {} is: {}'.format(object_id, data[0]['price']))

In [ ]:
#Definir variables
from datetime import datetime, timedelta, timezone

fecha_hora = datetime.strptime(data[0]['lastUpdate'],'%Y-%m-%dT%H:%M:%S.%f%z')
utc_5 = timezone(timedelta(hours=-5))
fecha_hora = fecha_hora.astimezone(utc_5)
precio = data[0]['price']
precio_apertura = data[0]['openPrice']
return_ytd = data[0]['totalReturnYtd']
precio_mas_alto = data[0]['highPrice']
precio_mas_bajo = data[0]['lowPrice']
precio_mas_alto_52_semanas = data[0]['highPrice52Week']
precio_mas_bajo_52_semanas = data[0]['lowPrice52Week']
cambio_precio_1_day = data[0]['priceChange1Day']
cambio_porcentual_1_day = data[0]['percentChange1Day']

In [ ]:
print(fecha_hora)

2022-05-31 13:29:48-05:00


In [ ]:
#Crear DataFrame
df = pd.DataFrame({
    "Precio":precio,
    "Precio de apertura":precio_apertura,
    "Precio más alto": precio_mas_alto,
    "Precio más bajo": precio_mas_bajo,
    "Precio más alto (52 semanas)": precio_mas_alto_52_semanas,
    "Precio más bajo (52 semanas)": precio_mas_bajo_52_semanas,
    "Return YTD": return_ytd,
    "Variación en soles": cambio_precio_1_day,
    "Variación porcentaje":cambio_porcentual_1_day,
}, index=[fecha_hora])
print(df.head())

In [ ]:
#Convertir a csv
df.to_csv('/content/tipo_cambio.csv', sep=';', encoding='utf-8-sig')